In [29]:
import os
import numpy as np
import json
import utils
from sklearn.model_selection import StratifiedKFold

In [30]:
# For reproducible results, set random_state to a number, otherwise set it to None
random_state = 42

## SELECT CATEGORY ##

# DATASET_NAME = "children"
# DATASET_NAME = "adolescents"
# DATASET_NAME = "eyesclosed"
# DATASET_NAME = "male"
# DATASET_NAME = "other"
DATASET_NAME = "all"


## SELECT DATASET PATH ##

# GRAPH_DIR_PREFIX = "./data/lanciano_datasets_corr_thresh_80/"
# DATA_DESCRIPTOR = "Lanciano-Processed"

GRAPH_DIR_PREFIX = "./data/generated_filt_global/pearson_corr_raw/"
DATA_DESCRIPTOR = "Raw-Correlation"


weighted = False
if DATA_DESCRIPTOR == "Raw-Correlation":
    weighted = True

In [31]:
A_GRAPH_DIR = f"{GRAPH_DIR_PREFIX}{DATASET_NAME}/asd/"
B_GRAPH_DIR = f"{GRAPH_DIR_PREFIX}{DATASET_NAME}/td/"
a_label="ASD"
b_label="TD"

In [32]:
# Read brain graph files into numpy arrays

if DATASET_NAME == "all":
    with open(GRAPH_DIR_PREFIX + "unique.json", "r") as fp:
        file_lists = json.load(fp)

    graphs_A = np.array([np.loadtxt(filename) for filename in file_lists[a_label]])
    graphs_B = np.array([np.loadtxt(filename) for filename in file_lists[b_label]])

else:
    graphs_A = utils.get_graphs_from_files(A_GRAPH_DIR)
    graphs_B = utils.get_graphs_from_files(B_GRAPH_DIR)

graphs, labels = utils.label_and_concatenate_graphs(graphs_A=graphs_A, graphs_B=graphs_B, a_label=a_label, b_label=b_label)

In [33]:
num_edges = 10

important_a_edge_matrix = np.zeros_like(graphs[0])
important_b_edge_matrix = np.zeros_like(graphs[0])

for i in range(10):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state + i)
    for train_index, test_index in folds.split(graphs, labels):
        train_graphs = graphs[train_index]
        train_labels = labels[train_index]

        # This is the algorithm used by DiscriminativeEdgesTransformer in the fit function

        # Create and Write Summary Graphs
        # Note that (u,v) is the same as (v,u), so we extract the upper triangle of the matrices
        summary_A = np.triu(utils.summary_graph(graphs[np.where(labels == a_label)]), k=1)
        summary_B = np.triu(utils.summary_graph(graphs[np.where(labels == b_label)]), k=1)
            
        # Get the difference network between the edge weights in group A and B
        diff_net = summary_A - summary_B

        # Find the num_edges most positive and most negative edge diffs
        partitions = np.argpartition(diff_net, (num_edges, -num_edges), axis=None)
        top_n = np.unravel_index(partitions[-num_edges:], diff_net.shape)
        bottom_n = np.unravel_index(partitions[:num_edges], diff_net.shape)

        # Ensure the top edges are all positive and the bottom edges are all negative
        top_edges = diff_net[top_n]
        positive = top_edges > 0
        positive_indices = (top_n[0][positive], top_n[1][positive])

        if len(positive_indices[0]) < num_edges:
            print(f"WARNING: only found {len(positive_indices)} positive DEs (looking for {num_edges}).")

        bottom_edges = diff_net[bottom_n]
        negative = bottom_edges < 0
        negative_indices = (bottom_n[0][negative], bottom_n[1][negative])

        if len(negative_indices[0]) < num_edges:
            print(f"WARNING: only found {len(negative_indices)} negative DEs (looking for {num_edges}).")

        important_a_edge_matrix[positive_indices] += diff_net[positive_indices]
        important_b_edge_matrix[negative_indices] += diff_net[negative_indices]

np.savetxt(f"./data/BrainNetViewer/DE_important_{a_label}_edges.edge", important_a_edge_matrix)
np.savetxt(f"./data/BrainNetViewer/DE_important_{b_label}_edges.edge", important_b_edge_matrix)
